In [1]:
from scipy.integrate import dblquad
import torch
import numpy as np
from arsf_envi_reader import envi_header
import shutil
import os

import json
import math
import affine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit



from tqdm import tqdm
# import multiprocess as mp
from numpy import trapz

In [2]:
torch.cuda.is_available()

True

In [3]:
get_ipython().run_line_magic('config', 'Completer.use_jedi = False')

In [4]:
in_header = envi_header.find_hdr_file(r"D:\wenqu\aviris\ang20190704t193319rfl\ang20190704t193319_rfl_v2v2_img.hdr")
header_data = envi_header.read_hdr_file(in_header)

In [5]:
# Get wavelengths and convert to NumPy array
low_res_wavelengths = header_data['wavelength'].split(',')
low_res_wavelengths = [float(w) for w in low_res_wavelengths]
low_res_wavelengths = np.array(low_res_wavelengths)


low_res_fwhm = header_data['fwhm'].split(',')
low_res_fwhm = [float(w) for w in low_res_fwhm]
low_res_fwhm = np.array(low_res_fwhm)

In [6]:
high_res_img = gdal.Open(r'D:\wenqu\site1b\uav\site1b_multi_or_radiance')
high_res_radiance = gdal.Open(r'D:\wenqu\site1b\uav\site1b_multi_or_radiance').ReadAsArray() 

high_res_radiance.shape

(273, 1022, 3256)

In [7]:
high_res_wavelength = [float(b.split(" ")[0]) for b in high_res_img.GetMetadata().values() if b != "nm"]
high_res_wavelength.sort()
high_res_wavelength = np.array(high_res_wavelength)

In [9]:
high_res_wavelength

array([ 398.697,  400.912,  403.127,  405.341,  407.556,  409.771,
        411.985,  414.2  ,  416.415,  418.63 ,  420.844,  423.059,
        425.274,  427.488,  429.703,  431.918,  434.133,  436.347,
        438.562,  440.777,  442.991,  445.206,  447.421,  449.636,
        451.85 ,  454.065,  456.28 ,  458.494,  460.709,  462.924,
        465.138,  467.353,  469.568,  471.783,  473.997,  476.212,
        478.427,  480.641,  482.856,  485.071,  487.286,  489.5  ,
        491.715,  493.93 ,  496.144,  498.359,  500.574,  502.789,
        505.003,  507.218,  509.433,  511.647,  513.862,  516.077,
        518.292,  520.506,  522.721,  524.936,  527.15 ,  529.365,
        531.58 ,  533.794,  536.009,  538.224,  540.439,  542.653,
        544.868,  547.083,  549.297,  551.512,  553.727,  555.942,
        558.156,  560.371,  562.586,  564.8  ,  567.015,  569.23 ,
        571.445,  573.659,  575.874,  578.089,  580.303,  582.518,
        584.733,  586.947,  589.162,  591.377,  593.592,  595.

In [10]:
import pandas as pd
high_res_irradiance = np.array(pd.read_csv('D:\wenqu\site1b\white_panel_radiance.csv')['irradiance'])
high_res_irradiance.shape

(273,)

In [8]:
def gauss(x, f, w):
    sigma = f/(2 * np.sqrt(2 *np.log(2)))
    y = np.exp((-(x-w)**2) / (2 * sigma**2))
    return y

In [11]:
high_res_img_flatten = high_res_radiance.reshape(273, -1)
high_res_img_flatten.shape

(273, 3327632)

In [ ]:
batch_pixels = []

with torch.no_grad():
    for i in tqdm(range(high_res_img_flatten.shape[1])):
        pixel_single = []
        nomins_integrals = [[f, w, quad(guass * high_res_img_flatten_tensor[:, i][None, ],398.697, 1001.1, args = (f, w))[0]] for f in low_res_fwhm for w in low_res_wavelengths]
        denomins_integrals = [[f, w, quad(guass * high_res_img_flatten_tensor[:, i][None, ],398.697, 1001.1, args = (f, w))[0]] for f in low_res_fwhm for w in low_res_wavelengths]